# LLM from scratch

Project based on: https://www.youtube.com/watch?v=UU1WVnMk4E8 

Dorothy and the Wizard in OZ soruce: https://gutenberg.org/cache/epub/22566/pg22566.txt

In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


hyperparameters

In [18]:
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

In [14]:
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text= f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [6]:
string_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_string = { i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')
print(encoded_hello)
decoded_hello = decode(encoded_hello)
print(decoded_hello)

[61, 58, 65, 65, 68]
hello


Using torch

In [7]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [8]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is", context, "target is", target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [10]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[30, 36, 45, 44, 44, 29, 42, 29],
        [ 0, 59, 71, 62, 58, 67, 57, 72],
        [ 1, 72, 61, 54, 69, 58,  9,  1],
        [65, 68, 71, 58,  0, 73, 61, 58]], device='cuda:0')
targets:
tensor([[36, 45, 44, 44, 29, 42, 29, 28],
        [59, 71, 62, 58, 67, 57, 72,  1],
        [72, 61, 54, 69, 58,  9,  1, 54],
        [68, 71, 58,  0, 73, 61, 58,  1]], device='cuda:0')


In [17]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            # batch size, time, channels (number of classes, vocabulary size)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


_co_Y4yVzBoA)gF)U8H9TLptk-]2DG6"2U"43ex0swbZn"hP"[Lgkyq3yL9P;BDY!Vl?9NzA!0t5Gv)CMHVpqhH?ToR H,ao3DaoVSW.spRk)5'UvfNY2Ho﻿O
u81tl6LW0:ksYeR&
Da"-LQtE*ve[M,fs[l69MMt4wg4SK8P;]C9-H(GbPu:7?TRJ-]IsQ":'X NA*(Ct4;tJS)wpKi-4:Wh6TqP50Y[7w"2
YaWRw50u&ove"S[lx;qfAx6UL)﻿EZqJ,WNvPKb1S _'(gMY!9 HQm5Wl S,fXlBe[7I!h8KW.Q-.ALd!ESv"[Rmxf96J!-2v'pv1swL7K]﻿c5YNl﻿jzmShPv'wvg4:VyCp8:&wDLmoUux;D3Y1Gp7Jdia;XaZawA)3jK5Rp4 ,
4nSn!S,X4*n,1J?"-v814SFAW.L5WetuNE
5Qx
he7xiRWOs-gbOSfU6﻿MDl9PhazAxRoSWoFaTdL﻿XO(XpvZNvFJivxG﻿!g4D


In [19]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    # if iter % eval_iters == 0:
    #     losses = estimate_loss()
    #     print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

4.324052333831787
